In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
import numpy as np
import importlib

# Juptyer magic: For export. Makes the plots size right for the screen 
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

%config InlineBackend.figure_formats = ['svg'] 


torch.backends.cudnn.deterministic = True
seed = np.random.randint(1,200)
# seed = 60 #59
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(seed)
g = torch.Generator()
g.manual_seed(seed)


# Data preparation

In [ ]:
cross_entropy = False #this leads to squared loss in the training
data_noise = 0.02
num_points = 5000
plotlim = [-3, 3]
subfolder = 'make_dataloader_1dim' #all the files generated from this notebook get saved into this subfolder

import os
if not os.path.exists(subfolder):
    os.makedirs(subfolder)


label = 'scalar' #choose between 'scalar' and 'vector'. Scalar means that the labels are -1 and 1, vector means that the labels are (0, -2) and (0, 2)



def create_dataloader(data_type, num_points = 3000, noise = 0.15, factor = 0.15, random_state = 1, shuffle = True, plotlim = [-2, 2], cross_entropy = False, label = 'scalar', ticks = True, markersize = 50, filename = 'trainingset'):
    
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_moons, make_circles, make_blobs
    from sklearn.preprocessing import StandardScaler
    import matplotlib.pyplot as plt
    from torch.utils import data as data
    from torch.utils.data import DataLoader, TensorDataset
    
    label_types = ['scalar', 'vector']
    if label not in label_types:
        raise ValueError("Invalid label type. Expected one of: %s" % label_types)
    
    if data_type == 'circles':
        X, y = make_circles(num_points, noise=noise, factor=factor, random_state=random_state, shuffle = shuffle)

    elif data_type == 'blobs':
        centers = [[-1, -1], [1, 1]]
        X, y = make_blobs(
    n_samples=num_points, centers=centers, cluster_std=noise, random_state=random_state)
           
    elif data_type == 'moons':
        X, y = make_moons(num_points, noise = noise, shuffle = shuffle , random_state = random_state)
    
    elif data_type == 'xor':
        X = torch.randint(low=0, high=2, size=(num_points, 2), dtype=torch.float32)
        y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0).float()
        # y = y.to(torch.int64)
        X += noise * torch.randn(X.shape)
        
    else: 
        print('datatype not supported')
        return None, None
    
    
    #Split the data into training and test set
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=random_state, shuffle = shuffle)
    
    # for plotting purposes
    data_0 = X_train[y_train == 0]
    data_1 = X_train[y_train == 1]
    
    plt.figure(figsize = (5,5), dpi = 100)
    plt.scatter(data_1[:, 0], data_1[:, 1], edgecolor="#333", alpha = 0.5, s = markersize)
    plt.scatter(data_0[:, 0], data_0[:, 1], edgecolor="#333",  alpha = 0.5, s = markersize)
    plt.xlim(plotlim)
    plt.ylim(plotlim)
    ax = plt.gca()
    ax.set_aspect('equal')
    plt.xlabel(r'$x_1$', fontsize=12)
    plt.ylabel(r'$x_2$', fontsize=12)
    if ticks == False:
        ax.set_xticks([])
        ax.set_yticks([])

    plt.savefig(filename + '.png', bbox_inches='tight', dpi=300, format='png', facecolor = 'white')
    plt.show()
    
    
    if label == 'vector':
        y_train = np.array([(0., -2.) if label == 1 else (0., 2.) for label in y_train])
        y_test = np.array([(0., -2.) if label == 1 else (0., 2.) for label in y_test])
    
    if label == 'scalar' and cross_entropy == False:
        y_train = np.array([1. if label == 1 else -1. for label in y_train])[:,None]  # unsqueeze to make it a 2D tensor for MSE
        y_test = np.array([1. if label == 1 else -1. for label in y_test])[:,None]  

    g = torch.Generator()
    g.manual_seed(random_state)
    
    # Convert to torch tensors
    X_train = torch.Tensor(X_train) # transform to torch tensor for dataloader
    y_train = torch.Tensor(y_train) #transform to torch tensor for dataloader

    X_test = torch.Tensor(X_test) # transform to torch tensor for dataloader
    y_test = torch.Tensor(y_test) #transform to torch tensor for dataloader

    if cross_entropy == True:
        X_train = X_train.type(torch.float32)  #type of orginial pickle.load data
        y_train = y_train.type(torch.int64) #dtype of original picle.load data

        X_test = X_test.type(torch.float32)  #type of orginial pickle.load data
        y_test = y_test.type(torch.int64) #dtype of original picle.load data
        
    else:
        X_train = X_train.type(torch.float32)  #type of orginial pickle.load data
        y_train = y_train.type(torch.float32) #dtype of original picle.load data

        X_test = X_test.type(torch.float32)  #type of orginial pickle.load data
        y_test = y_test.type(torch.float32) #dtype of original picle.load data


    train_data = TensorDataset(X_train,y_train) # create your datset
    test_data = TensorDataset(X_test, y_test)

    train = DataLoader(train_data, batch_size=64, shuffle=shuffle, generator=g)
    test = DataLoader(test_data, batch_size=256, shuffle=shuffle, generator = g) #128 before


    return train, test

dataloader, dataloader_viz = create_dataloader('moons', noise = data_noise, num_points = num_points, plotlim = plotlim, random_state = seed, label = label, cross_entropy=cross_entropy, filename = subfolder + '/trainingset')

for X, y in dataloader:
    print(y)
    break




## Model dynamics

In [ ]:
#Import of the model dynamics that describe the neural ODE
#The dynamics are based on the torchdiffeq package, that implements ODE solvers in the pytorch setting
from models.neural_odes import NeuralODEvar

#for neural ODE based networks the network width is constant. In this example the input is 2 dimensional
hidden_dim, data_dim = 2, 2 
augment_dim = 0
output_dim = 1

#T is the end time of the neural ODE evolution, time_steps are the amount of discretization steps for the ODE solver
T, time_steps = 10, 100 #
step_size = T/time_steps
num_params = 10 #the number of distinct parameters present in the interval. they are spread equidistant over the interval [0, T]. As there are 100 time_steps, the interval is divided into 10 parts, each of length 1 with 10 time_steps per subinterval.
bound = 0.
turnpike = False

non_linearity = 'tanh' #'relu' #
architecture = 'inside' #outside




## Training and generating level sets

In [ ]:

num_epochs = 60 #number of optimization runs in which the dataset is used for gradient decent

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
anode = NeuralODEvar(device, data_dim, hidden_dim, output_dim = output_dim, augment_dim=augment_dim, non_linearity=non_linearity, 
                    architecture=architecture, T=T, time_steps=time_steps, num_params = num_params, cross_entropy=cross_entropy)


optimizer_anode = torch.optim.Adam(anode.parameters(), lr=1e-3) 

In [ ]:
from models.training import doublebackTrainer

trainer_anode = doublebackTrainer(anode, optimizer_anode, device, cross_entropy=cross_entropy, turnpike = turnpike,
                         bound=bound, verbose = True) 
trainer_anode.train(dataloader, num_epochs)

In [ ]:
x1 = torch.arange(0, 1, step=0.5)
x2 = torch.arange(0, 1, step=0.5)
xx1, xx2  = torch.meshgrid(x1, x2)  # Meshgrid function as in numpy should emulate indexing='xy'
model_inputs = torch.stack([xx1, xx2], dim=-1)

print(model_inputs)

preds, _ = anode(model_inputs)

print(preds.sigmoid().squeeze())

In [ ]:
test = torch.tensor([1., 0., 7])
test[-1] = test[-1].sigmoid()
test


In [ ]:
import plots.plots
importlib.reload(plots.plots)
from plots.plots import classification_levelsets
classification_levelsets(anode)
plt.plot(trainer_anode.histories['epoch_loss_history'])
plt.xlim(0, len(trainer_anode.histories['epoch_loss_history']) - 1)
plt.ylim(0)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:

import plots.gifs
importlib.reload(plots.gifs) # Reload the module to ensure the latest changes are applied
from plots.gifs import trajectory_gif
from IPython.display import Image

#the function trajectory_gif creates the evolution trajectories of the input data through the network
#the passing time corresponds to the depth of the network

for X_viz, y_viz in dataloader_viz:
    print(y_viz.shape)
    trajectory_gif(anode, X_viz[0:50], y_viz[0:50], timesteps=time_steps, filename = subfolder + '/trajectory', axlim = 8, dpi = 100)
    break

#Display of the generated gif
traj = Image(filename=subfolder + "/trajectory.gif")
display(traj)

In [ ]:
importlib.reload(plots.plots)

import os

from plots.plots import plot_points_from_traj, input_to_traj_and_color, plot_vectorfield
from plots.plots import create_gif_subintervals
import imageio
import io

filename = subfolder + '/FTLE_per_param_fun_test'
create_gif_subintervals(anode, le_density=2, filename=filename)
LEevo_test = Image(filename=filename + ".gif")
display(LEevo_test)